# Extraction and cleaning of Inside Airbnb data
#### Source: Inside Airbnb Portal
#### City: New York City, New York, United States
#### As of: 05 June, 2023 (Data updated every quarter)

In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats as st
import sqlite3
import csv

In [2]:
# Store filepath into variable
listings = Path("Resources/listings.csv")

In [3]:
# Read in file
listings_df = pd.read_csv(listings)
listings_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,5121,Rental unit in Brooklyn · ★4.52 · 1 bedroom · ...,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.29,2,0,0,NaN
1,2595,Rental unit in New York · ★4.68 · Studio · 1 b...,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,240,30,49,2022-06-21,0.30,3,325,1,NaN
2,14991,Rental unit in New York · ★4.93 · 1 bedroom · ...,59023,Bianca,Manhattan,Lower East Side,40.72207,-73.98976,Private room,120,5,22,2023-05-14,0.14,1,268,3,NaN
3,5136,Rental unit in Brooklyn · ★5.0 · 2 bedrooms · ...,7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,276,21,3,2022-08-10,0.03,1,275,1,NaN
4,59709,Rental unit in New York · ★4.77 · 2 bedrooms ·...,186084,Ricardo & Ashlie,Manhattan,Chinatown,40.71780,-73.99320,Entire home/apt,325,7,69,2023-04-28,0.47,1,38,23,NaN


In [4]:
# Extract the rating from the column "name" and create a new column named "rating"
listings_df['rating'] = listings_df['name'].str.extract('(★(?:\d+\.\d+|New))')
listings_df['rating'] = listings_df['rating'].str.extract('((?:\d+\.\d+|New))')
listings_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,rating
0,5121,Rental unit in Brooklyn · ★4.52 · 1 bedroom · ...,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.29,2,0,0,NaN,4.52
1,2595,Rental unit in New York · ★4.68 · Studio · 1 b...,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,240,30,49,2022-06-21,0.30,3,325,1,NaN,4.68
2,14991,Rental unit in New York · ★4.93 · 1 bedroom · ...,59023,Bianca,Manhattan,Lower East Side,40.72207,-73.98976,Private room,120,5,22,2023-05-14,0.14,1,268,3,NaN,4.93
3,5136,Rental unit in Brooklyn · ★5.0 · 2 bedrooms · ...,7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,276,21,3,2022-08-10,0.03,1,275,1,NaN,5.0
4,59709,Rental unit in New York · ★4.77 · 2 bedrooms ·...,186084,Ricardo & Ashlie,Manhattan,Chinatown,40.71780,-73.99320,Entire home/apt,325,7,69,2023-04-28,0.47,1,38,23,NaN,4.77


In [5]:
# Get the column names. 
listings_df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license', 'rating'],
      dtype='object')

In [6]:
# Keep and reorder columns we're interested in
listings_df=listings_df[["id", "room_type", "price", "minimum_nights", "rating", "number_of_reviews", "availability_365",
                         "neighbourhood_group", "neighbourhood", "latitude", "longitude"]]
listings_df.head()

,id,room_type,price,minimum_nights,rating,number_of_reviews,availability_365,neighbourhood_group,neighbourhood,latitude,longitude
0,5121,Private room,60,30,4.52,50,0,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512
1,2595,Entire home/apt,240,30,4.68,49,325,Manhattan,Midtown,40.75356,-73.98559
2,14991,Private room,120,5,4.93,22,268,Manhattan,Lower East Side,40.72207,-73.98976
3,5136,Entire home/apt,276,21,5.0,3,275,Brooklyn,Sunset Park,40.66265,-73.99454
4,59709,Entire home/apt,325,7,4.77,69,38,Manhattan,Chinatown,40.71780,-73.99320


In [7]:
# Dataframe count and number of NA values
print(f"Number of rows: \n{listings_df.count()}")
print("\n")
print(f"Number of NA values: \n{listings_df.isna().sum()}")

Number of rows: 
id                     43566
room_type              43566
price                  43566
minimum_nights         43566
rating                 28376
number_of_reviews      43566
availability_365       43566
neighbourhood_group    43566
neighbourhood          43566
latitude               43566
longitude              43566
dtype: int64


Number of NA values: 
id                         0
room_type                  0
price                      0
minimum_nights             0
rating                 15190
number_of_reviews          0
availability_365           0
neighbourhood_group        0
neighbourhood              0
latitude                   0
longitude                  0
dtype: int64


In [8]:
# Replace NA and 'New' values in the column "rating" by 0
# # Drop null values
listings_df["rating"].fillna(0, inplace=True)
listings_df["rating"] = listings_df["rating"].replace("New", "0")
listings_df = listings_df.dropna()
print(f"Number of rows: \n{listings_df.count()}")
print("\n")
print(f"Number of NA values: \n{listings_df.isna().sum()}")

Number of rows: 
id                     43566
room_type              43566
price                  43566
minimum_nights         43566
rating                 43566
number_of_reviews      43566
availability_365       43566
neighbourhood_group    43566
neighbourhood          43566
latitude               43566
longitude              43566
dtype: int64


Number of NA values: 
id                     0
room_type              0
price                  0
minimum_nights         0
rating                 0
number_of_reviews      0
availability_365       0
neighbourhood_group    0
neighbourhood          0
latitude               0
longitude              0
dtype: int64


In [9]:
# Get the data types
listings_df.dtypes

id                       int64
room_type               object
price                    int64
minimum_nights           int64
rating                  object
number_of_reviews        int64
availability_365         int64
neighbourhood_group     object
neighbourhood           object
latitude               float64
longitude              float64
dtype: object

In [10]:
# convert 'rating' to float
listings_df["rating"] = listings_df["rating"].astype(float)
listings_df.dtypes

id                       int64
room_type               object
price                    int64
minimum_nights           int64
rating                 float64
number_of_reviews        int64
availability_365         int64
neighbourhood_group     object
neighbourhood           object
latitude               float64
longitude              float64
dtype: object

In [11]:
# Rename columns
listings_df = listings_df.rename(
    columns={"price": "price_per_night",
            "availability_365": "availability"})
listings_df.head()

,id,room_type,price_per_night,minimum_nights,rating,number_of_reviews,availability,neighbourhood_group,neighbourhood,latitude,longitude
0,5121,Private room,60,30,4.52,50,0,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512
1,2595,Entire home/apt,240,30,4.68,49,325,Manhattan,Midtown,40.75356,-73.98559
2,14991,Private room,120,5,4.93,22,268,Manhattan,Lower East Side,40.72207,-73.98976
3,5136,Entire home/apt,276,21,5.00,3,275,Brooklyn,Sunset Park,40.66265,-73.99454
4,59709,Entire home/apt,325,7,4.77,69,38,Manhattan,Chinatown,40.71780,-73.99320


In [12]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of price for each room_type

# Use groupby and summary statistical methods to calculate the following properties of each room_type:
# mean, median, variance, standard deviation, and SEM of the price.
cdf_mean = listings_df.groupby("room_type")["price_per_night"].mean()
cdf_median = listings_df.groupby("room_type")["price_per_night"].median()
cdf_variance = listings_df.groupby("room_type")["price_per_night"].var()
cdf_std = listings_df.groupby("room_type")["price_per_night"].std()
cdf_sem = listings_df.groupby("room_type")["price_per_night"].sem()

# Assemble the resulting series into a single summary DataFrame.
summary_stats = {"Mean Price": cdf_mean, "Median Price": cdf_median, "Price Variance": cdf_variance,
                "Price Std. Dev.": cdf_std, "Price Std. Err.": cdf_sem}
stats_df = pd.DataFrame(summary_stats)
stats_df

,Mean Price,Median Price,Price Variance,Price Std. Dev.,Price Std. Err.
room_type,,,,,
Entire home/apt,273.369062,190.0,3.072098e+05,554.265069,3.530351
Hotel room,483.615894,410.0,1.266816e+05,355.923641,28.964655
Private room,160.973166,79.0,1.172609e+06,1082.870738,8.029861
Shared room,125.874138,60.0,2.418108e+05,491.742666,20.418503


In [13]:
# output cleaned file to csv
listings_df.to_csv("Outputs/inside_airbnb_nyc_cleaned.csv", index = False)

# Database Creation using SQLITE
####  - Create a sqlite Database names "data"
####  - Create a table named "listings"
####  - Import the CSV into the DataBase

In [16]:
# Connect to or create the database
conn = sqlite3.connect('data.db')

# Create a table named 'Listings'
conn.execute('''
    CREATE TABLE listings (
        id INT,
        room_type VARCHAR(50),
        price_per_night INT,
        minimum_nights INT,
        rating FLOAT,
        number_of_reviews INT,
        availability INT,
        neighbourhood_group VARCHAR(50),
        neighbourhood VARCHAR(80),
        latitude FLOAT,
        longitude FLOAT
    )
''')

In [17]:
conn = sqlite3.connect('data.db')

# Open the CSV file and read data
with open('Outputs/inside_airbnb_nyc_cleaned.csv', 'r', encoding='UTF-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip header if present

    # Iterate through rows and insert data
    for row in csv_reader:
        conn.execute('''
            INSERT INTO listings (id, room_type, price_per_night, minimum_nights, rating, number_of_reviews, availability, neighbourhood_group, neighbourhood, latitude, longitude)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', row)

# Commit changes and close the connection
conn.commit()
conn.close()